#### Usefull websites:

Create a rotating proxy crawler in Python 3 https://codelike.pro/create-a-crawler-with-rotating-ip-proxy-in-python/  
Advanced Scraping - Form Submission http://jonathansoma.com/lede/foundations-2017/classes/adv-scraping/advanced-scraping-form-submission/  
Web Scraping 201: finding the API http://www.gregreda.com/2015/02/15/web-scraping-finding-the-api/

In [1]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import random
import requests

In [2]:
bosch_params = {'searchTerm':'thesis'}
daimler_params = {_: 1555346176442}

In [3]:
# returns a request object from given url with random user agent and random proxy
def request(url:str,request_type='get',post_params={}) -> requests.models.Response:
    
    # check parameter requirements
    if request_type != 'get' and request_type != 'post':
        raise ValueError("request_type must be 'get' or 'post' of type str")
        
    if type(post_params) is not dict:
        raise ValueError("post_params must be of type dict()")
    
    #  disable ANY annoying exception with a fallback to default user agent if anything goes wrong
    ua = UserAgent(fallback = 'Mozilla/5.0 (Windows; U; Windows NT 6.1; zh-HK) AppleWebKit/533.18.1 (KHTML, like Gecko) Version/5.0.2 Safari/533.18.5')

    # update saved database of user agents
    # fake_useragent stores collected data at os temp dir
    print('update user agent database')
    try: 
        ua.update()
    except:
        print('user agent database update failed')
    
    print('get proxy list')
    # will contain proxies [ip, port]
    proxies = []

    # retrieve latest proxies from website
    proxies_req = requests.get(url='https://www.sslproxies.org/', headers={'user-agent': ua.random})
    proxies_req.encoding = 'utf-8'

    # create beatifulsoup instance, parse html and get section with proxy list of website
    parsed_html = BeautifulSoup(proxies_req.text, 'html.parser')
    proxies_table = parsed_html.find(id='proxylisttable')

    # save proxies from proxy list of website in the array
    for row in proxies_table.tbody.find_all('tr'):
        proxies.append({
        'ip':   row.find_all('td')[0].string,
        'port': row.find_all('td')[1].string
      })
        
    # retrieve a random index proxy
    def random_proxy_idx():
        return random.randint(0, len(proxies) - 1)
    
    print('try proxies')
    # try if proxy server is running, if not try other random proxy and user agent, try maximum x times then exit loop
    proxy_is_good = False
    i = 0
    x = 10
    while not proxy_is_good:
    
        # get dict for random proxy 
        random_proxy = proxies[random_proxy_idx()]
        random_proxy_dict = {'http': 'http://' + random_proxy['ip'] + ':' + random_proxy['port'],
                        'https': 'https://' + random_proxy['ip'] + ':' + random_proxy['port']}
        # select random user agent
        user_agent = ua.random
    
        # try to get request object from url with random user agent and random proxy
        try:
            if request_type == 'get':
                req = requests.get(url=url, headers={'user-agent': user_agent},proxies=random_proxy_dict,timeout=1)
            elif request_type == 'post':
                req = requests.post(url=url, headers={'user-agent': user_agent},proxies=random_proxy_dict,data=post_params,timeout=1)
            else:
                raise ValueError("request_type must be 'get' or 'post' of type str")
                
        # if anything goes wrong try again with differnt user agent and proxy
        except:
            print('ERROR','Proxy:',random_proxy,'User Agent:',user_agent)
            if i == x:
                print('All',i,'proxies are not reachable')
                break
            i += 1
        # if everything goes right, end loop and return request object
        else:
            proxy_is_good = True
            print('SUCCESS','Proxy:',random_proxy,'User Agent:',user_agent)
    return req

In [4]:
# Bosch requires a get request and blocks post requests
bosch_req = request('https://www.bosch.de/karriere/jobs/', request_type='get', post_params=bosch_params)

update user agent database
get proxy list
try proxies
ERROR Proxy: {'ip': '101.51.141.46', 'port': '37858'} User Agent: Mozilla/5.0 (iPad; CPU OS 5_1 like Mac OS X) AppleWebKit/534.46 (KHTML, like Gecko ) Version/5.1 Mobile/9B176 Safari/7534.48.3
SUCCESS Proxy: {'ip': '92.114.234.206', 'port': '38713'} User Agent: Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_6; zh-cn) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.4 Safari/533.20.27


In [5]:
# Daimler uses an API to send the data to the client.
# The Daimler Carrer Website itself is just a HTML template without data.
# Data and template render client side and are sent seperately to the client.
# In this case we must find the API URL that returns the raw data (the Daimler API sends JSON).
daimler_api_url = 'https://global-jobboard-api.daimler.com/v3/search/%7B%22LanguageCode%22%3A%22EN%22%2C%22ScoreThreshold%22%3A80%2C%22SearchParameters%22%3A%7B%22MatchedObjectDescriptor%22%3A%5B%22PositionID%22%2C%22PositionTitle%22%2C%22PositionURI%22%2C%22LogoURI%22%2C%22OrganizationName%22%2C%22Organization%22%2C%22Organization.MemberCode%22%2C%22ParentOrganization%22%2C%22PositionLocation.CityName%22%2C%22PositionLocation.Longitude%22%2C%22PositionLocation.Latitude%22%2C%22PositionIndustry.Name%22%2C%22JobCategory.Name%22%2C%22JobCategory.Code%22%2C%22CareerLevel.Name%22%2C%22CareerLevel.Code%22%2C%22Facet%3AParentOrganization%22%2C%22Facet%3APositionLocation.CityName%22%2C%22Facet%3APositionLocation.CountryName%22%2C%22PublicationStartDate%22%2C%22ParentOrganizationGenesisID%22%5D%2C%22FirstItem%22%3A0%2C%22CountItem%22%3A20%7D%2C%22SearchCriteria%22%3A%5B%7B%22CriterionName%22%3A%22PositionFormattedDescription.Content%22%2C%22CriterionValue%22%3A%5B%22data%22%5D%7D%2C%7B%22CriterionName%22%3A%22PublicationLanguage.Code%22%2C%22CriterionValue%22%3A%5B%22EN%22%5D%7D%2C%7B%22CriterionName%22%3A%22PublicationChannel.Code%22%2C%22CriterionValue%22%3A%5B%2212%22%5D%7D%2C%7B%22CriterionName%22%3A%22CareerLevel.Code%22%2C%22CriterionValue%22%3A%5B%2240%22%5D%7D%5D%7D?_=1555346176443'
daimler_req = request(daimler_api_url, request_type='post', post_params=daimler_params)

update user agent database
get proxy list
try proxies
ERROR Proxy: {'ip': '121.101.190.246', 'port': '61641'} User Agent: Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.67 Safari/537.36
ERROR Proxy: {'ip': '186.43.33.81', 'port': '35736'} User Agent: Mozilla/5.0 (Windows NT 10.0; WOW64; rv:45.66.18) Gecko/20177177 Firefox/45.66.18
ERROR Proxy: {'ip': '91.218.163.74', 'port': '50661'} User Agent: Mozilla/5.0 (Windows; U; Windows NT 6.1; cs-CZ) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.4 Safari/533.20.27
SUCCESS Proxy: {'ip': '180.183.122.147', 'port': '8213'} User Agent: Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.3319.102 Safari/537.36


In [6]:
daimler_data = daimler_req.json()
bosch_data = BeautifulSoup(bosch_req.text, 'html.parser')